# Building a Spam Filter with Naive Bayes

This project is part of guided projects available on [Dataquest.io](https://dataquest.io). The aim of this project is to use naive Bayes algorithm to mark SMS messages as spam or not spam. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). Data collection process can be found following [this link](https://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition).

There are two colums in the dataset. First marking the message as:
- `ham` – no spam
- `spam` - spam

## 1. Exploring the Dataset

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import re

In [2]:
# opening the SMS Spam Collection .csv file
sms_spam = pd.read_csv("SMSSpamCollection.csv", sep="\t", header=None, names=["label", "SMS"])

In [3]:
# exploring the dataset
print(sms_spam.info())
print("\n")
print(sms_spam.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
label    5572 non-null object
SMS      5572 non-null object
dtypes: object(2)
memory usage: 87.2+ KB
None


  label                                                SMS
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [4]:
sms_spam["label"].value_counts(normalize=True) * 100

ham     86.593683
spam    13.406317
Name: label, dtype: float64

The dataset contains two colums and 5572 rows, all non-null values. 13% of the messages are marked as spam.

## 2. Training and Test Set
To test the spam filter, we're first going to split our dataset into two categories **training set** and **testing set.** We're going to keep 80% of our dataset for training, and 20% for testing (we want to train the algorithm on as much data as possible, but we also want to have enough test data).

|Set    |No. Messages|Percentage|
|:------| ----------:|---------:|
|Trainig| 4,458      | 80 %     |
|Testing| 1,114      | 20 %     |

For this project, our goal is to create a spam filter that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).


In [5]:
# randomizing the entire dataset
sms_random = sms_spam.sample(frac=1, random_state=1)

# spliting the datset into train and test
train = sms_random.iloc[0:4458].reset_index(drop=True)
test = sms_random.iloc[4458:].reset_index(drop=True)

# checking the leng of bot sets
print(train.shape)
print(test.shape)

(4458, 2)
(1114, 2)


In [6]:
# checking the spam percentage in bot sets - should resemble the original
print(train["label"].value_counts(normalize=True) * 100)
print("\n")
print(test["label"].value_counts(normalize=True) * 100)

ham     86.54105
spam    13.45895
Name: label, dtype: float64


ham     86.804309
spam    13.195691
Name: label, dtype: float64


Both training and test set have similar spam and not spam messages ratio. 

## 3. Data Cleaning
To calculate probabilities that a unique word might be potentially spammy (words like money, win, secret), we first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need. Right now, our training and test sets have this format (simplifeid):

|index|label|sms                                     |
|:---:|----:| -------:                               |
| 0   |spam | SECRET PRIZE! CLAIM SECRET PRIZE NOW!! |
| 1   | ham | Comming to my secret party?            |
| 2   |spam | Winner! Claim secret prize now         |

To make the calculations easier, we want bring the data to this format (the table below is a transformation of the table you see above):


|index|label|secret|prize|claim|now|comming|to|my|party|winner|
|:---:|----:|-----:|-----:|-----:|---:|-------:|---:|----:|----:|---:|
|0|spam|2|2|1|1|0|0|0|0|0|
|1|ham|1|0|0|0|1|1|1|1|0|
|2|spam|1|1|1|1|0|0|0|0|1|

About the transformation above, notice that:

- The `SMS` column doesn't exist anymore.
- Instead, the `SMS` column is replaced by a series of new columns, where each column represents a unique word from the vocabulary.
- Each row describes a single message. For instance, the first row corresponds to the message *"SECRET PRIZE! CLAIM SECRET PRIZE NOW!!",* and it has the values `spam, 2, 2, 1, 1, 0, 0, 0, 0, 0`. These values tell us that:
    - The message is spam.
    - The word "secret" occurs two times inside the message.
    - The word "prize" occurs two times inside the message.
    - The word "claim" occurs one time inside the message.
    - The word "now" occurs one time inside the message.
    - The words "coming", "to", "my", "party", and "winner" occur zero times inside the message.
- All words in the vocabulary are in lower case, so "SECRET" and "secret" come to be considered to be the same word.
- Punctuation is not taken into account anymore (for instance, we can't look at the table and conclude that the first message initially had three exclamation marks).

In [7]:
# training set before cleaning
train.head()

,label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [8]:
# removing non aA-zZ and 0-9 characters
train["SMS"] = train["SMS"].str.replace("\W", " ",)

# lowering all charasters
train["SMS"] = train["SMS"].str.lower()

# training set after cleaning
train.head()

,label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


### 3.2 Creating the Vocabulary

In [9]:
# empty vocabulary list
vocabulary = []

# transforming each message into list of words
train["SMS"] = train["SMS"].str.split()

# iterating over the SMS column and appending unique words to vocublary list
for message in train["SMS"]:
    for word in message:
        vocabulary.append(word)

# transforming vocabulary into set, removing duplicates and back to a list
vocabulary = list(set(vocabulary))

# lenght of the vocabulary list
len(vocabulary)

7783

From the train set we have now a list of 7,783 unique words.

#### 3.3 Final Training Set
Let's now move to creating the vocabulary, which in this context means a list with all the unique words in our training set.

In [10]:
word_counts_per_sms = {unique_word: [0] * len(train["SMS"]) for unique_word in vocabulary}

for index, sms in enumerate(train["SMS"]):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [11]:
# transforming the word counts per sms to dataframe
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,spoken,4get,fundamentals,may,w1,pie,nickey,done,mad,no,...,1x150p,bcz,gods,tue,5k,gower,skills,1mega,2wt,anjie
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# combining the training set with the table above
train_clean = pd.concat([train, word_counts], axis=1)
train_clean.head()

,label,SMS,spoken,4get,fundamentals,may,w1,pie,nickey,done,...,1x150p,bcz,gods,tue,5k,gower,skills,1mega,2wt,anjie
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 4. Calculating Constant First

To calculate the Naive Bayes algorithm will need to know the probability values of the two equations below to be able to classify new messages:

- P(*Spam* | *w<sub>1</sub>, w<sub>2</sub>,..., w<sub>n</sub>*) ∝ *P(Spam)* ⋅ ∏<sup>n</sup> <sub>i=1</sub> *P(w<sub>i</sub> | Spam)*
- P(*Ham* | *w<sub>1</sub>, w<sub>2</sub>,..., w<sub>n</sub>*) ∝ *P(Ham)* ⋅ ∏<sup>n</sup> <sub>i=1</sub> *P(w<sub>i</sub> | Ham)*

Also, to calculate P(w<sub>i</sub> | Spam) and P(w<sub>i</sub> | Ham) inside the formulas above, we need to use these equations:

- P(w<sub>i</sub> | Spam) = (*N<sub>wi | Spam</sub> + α*) / (*N<sub>Spam</sub> + α ⋅ N<sub>Vocabulary</sub>*)
- P(w<sub>i</sub> | Ham) = (*N<sub>wi | Ham</sub> + α*) / (*N<sub>Ham</sub> + α ⋅ N<sub>Vocabulary</sub>*)

Let's first calculate:
- P(Spam) and P(Ham)
- N<sub>Spam</sub>, N<sub>Ham</sub>, N<sub>Vocabulary</sub>

To use Laplace smoothing we'll set α = 1.

### 4.1 P(Spam) and P(Ham)

In [13]:
# isolating spam messages and ham messages
spam = train_clean[train_clean["label"] == "spam"]
ham = train_clean[train_clean["label"] == "ham"]

# calculating probabilities of spam and ham
p_spam = len(spam) / len(train_clean)
p_ham = len(ham) / len(train_clean)

### 4.2 N<sub>Spam</sub>, N<sub>Ham</sub> and N<sub>Vocabulary</sub>

In [14]:
# number of words in spam messages
n_words_per_spam_message = spam["SMS"].apply(len)
n_spam = n_words_per_spam_message.sum()

# number of words in ham messages
n_words_per_ham_message = ham["SMS"].apply(len)
n_ham = n_words_per_ham_message.sum()

# number of words in vocabulary messages
n_vocabulary = len(vocabulary)

# alpha variable
alpha = 1

## 5. Calculating Parametres
We have 7,783 words in our `vocabulary`, which means we'll need to calculate a total of 15,566 probabilities. For each word, we need to calculate both P(w<sub>i</sub> | Spam) and P(w<sub>i</sub> | Ham).

In [15]:
# initializing dictionary for each word in voacabulary for spam and ham
spam_dic = {unique_word:0 for unique_word in vocabulary}
ham_dic = {unique_word:0 for unique_word in vocabulary}

# calculating the probabilites for each word
for word in vocabulary:
    n_word_given_spam = spam[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha * n_vocabulary)
    spam_dic[word] = p_word_given_spam

    n_word_given_ham = ham[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha * n_vocabulary)
    ham_dic[word] = p_word_given_ham


## 6. Classifing New Messages
All the preparation for creating the spam filter are done and now let's create a function that will:
- take as an input new message
- Compares the values of P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), and:
    - If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) > P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as ham.
    - If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) < P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as spam.
    - If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) = P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the algorithm may request human help.

In [16]:
def classify(message):

    message = re.sub("\W", " ", message)    # cleaning the input message
    message = message.lower()
    message = message.split()
    
    p_spam_given_message = p_spam           # initial probability value set as the train dataset probability
    p_ham_given_message = p_ham

    for word in message:                   # fininding word in both dictionaries
        if word in spam_dic:
            p_spam_given_message *= spam_dic[word]   # multiplying the word probabilities to get the final probability
            
        if word in ham_dic:
            p_ham_given_message *= ham_dic[word]
        # ignoring words that are not part of the vocabulary

    print("P(Spam|message):", p_spam_given_message)
    print("P(Ham|message):", p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print("Label: Ham")
    elif p_ham_given_message < p_spam_given_message:
        print("Label: Spam")
    else:
        print("Equal proabilities, have a human classify this!")

### 6.1 Initial Spam Filter Testing

In [17]:
# testing messages
test_spam_message = "WINNER!! This is the secret code to unlock the money: C3421."
test_ham_message = "Sounds good, Tom, then see u there"

In [18]:
# classifing test spam message
classify(test_spam_message)

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [19]:
# clasifing test ham message
classify(test_ham_message)

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## 7. Measuring Spam Filter Accuracy
We we aiming for 80% spam filter accurancy. To see whether we've achieved that let's test our spam filter in the testing set of 1,114 messages. To achieve 80% accurancy we need to correctly mark 891 messages. To mark the messages correctly we will slighlty adjust the `classify` function so it labels the messages rather than printing the outcome. We then need to add the prediction the testing set as a column.

In [20]:
# adjusting the classify function
def classify_test_set(message):

    message = re.sub("\W", " ", message)
    message = message.lower()
    message = message.split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in spam_dic:
            p_spam_given_message *= spam_dic[word]
            
        if word in ham_dic:
            p_ham_given_message *= ham_dic[word]

    if p_ham_given_message > p_spam_given_message:
        return "ham"
    elif p_ham_given_message < p_spam_given_message:
        return "spam"
    else:
        return "human"

In [21]:
# using the function to create new column in the test set
test["prediction"] = test["SMS"].apply(classify_test_set)
test.head()

,label,SMS,prediction
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


### 7.1 Accurancy

In [22]:
correct = 0
total = len(test)

for row in test.iterrows():
    row = row[1]
    if row["label"] == row["prediction"]:
        correct += 1

print("Correct: ", correct)
print("Incorect:", total - correct)
print("Total:", total)
print("Accurancy: ", round((correct/total) * 100, 2), "%")

Correct:  1100
Incorect: 14
Total: 1114
Accurancy:  98.74 %


The Spam filter we've build has 98.74 % accurancy. The filter looked at 1,114 messages and marked 1,100 correctly.